In [18]:
import numpy as np
from simulator.csad import CSAD_DP_6DOF
from Observer.EKF import EKF


dt = 0.01
N = 100

system = CSAD_DP_6DOF(dt)
KalmanFilter = EKF(dt, system._M, system._D)


storage_state = np.zeros((N,7))
storage_observer = np.zeros((N, 15))

x = np.zeros(15)

for i in range(N):
    # time
    t = (i+1)*system._dt

    # Thruster forces
    tau = np.array([100,0,0,0,0,0])

    # Env. forces
    # =0

    # Calculate x_dot and integrate
    system.x_dot(0,0,tau)
    system.integrate()

    # Observer
    xdot = KalmanFilter.state_function(x, np.array([100,0,0]), np.zeros(6))
    x = x + KalmanFilter._dt * xdot
    # Save for storage
    storage_state[i] = np.concatenate([t, system.get_eta()], axis=None)
    storage_observer[i] = x

print(KalmanFilter._Minv)
print(system._M)
#print(storage_state[:,1])
print('breaknjnj')
print(storage_observer[:,6])

C:\Users\Haral\OneDrive\Dokumenter\NTNU\9. semester\Prosjektoppgave\Simuleringsmodell_med_Jan\Wave-Model\Observer\EKF.py:95: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  Ax1 = np.array(


[[ 0.00691132  0.          0.        ]
 [ 0.          0.00382175 -0.00024328]
 [ 0.         -0.00024328  0.00878788]]
[[144.69021416   0.           0.           0.           0.
    0.        ]
 [  0.         262.12245      0.          -0.81381512   0.
    7.256382  ]
 [  0.           0.         320.83431      0.          24.88125
    0.        ]
 [  0.          -0.81381512   0.           4.5040458    0.
   -0.67311978]
 [  0.           0.          24.88125      0.         146.402805
    0.        ]
 [  0.           7.256382     0.          -0.67311978   0.
  113.993996  ]]
breaknjnj
[0.00000000e+00 6.91131744e-05 2.07114902e-04 4.13781292e-04
 6.88889180e-04 1.03221613e-03 1.44354042e-03 1.92264107e-03
 2.46929779e-03 3.08329102e-03 3.76440192e-03 4.51241235e-03
 5.32710489e-03 6.20826281e-03 7.15567010e-03 8.16911145e-03
 9.24837223e-03 1.03932385e-02 1.16034972e-02 1.28789356e-02
 1.42193419e-02 1.56245050e-02 1.70942144e-02 1.86282604e-02
 2.02264339e-02 2.18885264e-02 2.36143301e-0